<center><b><font style="color: orange" size="5">BE-5: Thresholding and Binary Morphology</font></b></center>
&ensp;

> <left><b><font style="color: blue" size="5"> Deadline: </font></b></center> &ensp;  <left><b><font style="color: red" size="5"> **26/03/2024 latest at 23h59**</font></b></center> &ensp;

## Enter your name here: (double click on this cell to fill the information)
----------------

> Student 1:
>> First Name:

>> Last Name:
----------------------------------
> Student 2:
>> First Name:

>> Last Name:

<center><b><font style="color: orange" size="5">Objectives CTD-5</font></b></center>
&ensp;

1. Evaluate the performance of various Thresholding Algorithm and their robustness against noise.
2. Understand the importance of some morphological operator and how can be used to pre- or post process images.

# Packages

- Please make sure the following **packages** can be imported: 
    - `numpy`
    - `matplotlib`
    - `skimage`
    - `cv2`
    
- Otherwise you can **install them directly** via jupyter notebook here via:
    - create a new cell using the plus sign + up in the bar
    - use the command: `!pip install package_name` (e.g `!pip install skimage`) 
    - Note that in some computers, you may need to restart the notebook/kernal to take effect. For that, in the toolbar go to kernel, press on restart or restart & clear out.
    

In [ ]:
# Please make sure the following packages can be imported: numpy, matplotlib, scipy, cv2 and skimage
import numpy as np
import matplotlib.pyplot as plt
from skimage import io as skio
import skimage.morphology as morph
from skimage.draw import line
import random
import cv2
from skimage.filters import threshold_otsu,threshold_multiotsu,gaussian

### Useful functions to be used during the session

In [ ]:
def stucturing_element(forme,taille,angle=45):
    """returns a structuring element in the shape  
     -diamond 
     -disk' 
     -ball 
     -square' of dimension size (it is better to use size=odd)
     -line' segment of length size and orientation angle (between 0 and 180 degrees)
    """

    if forme == 'diamond':
        return morph.diamond(taille)
    if forme == 'disk':
        return morph.disk(taille)
    if forme == 'square':
        return morph.square(taille)
    if forme == 'line':
        angle=int(-np.round(angle))
        angle=angle%180
        angle=np.float32(angle)/180.0*np.pi
        x=int(np.round(np.cos(angle)*taille))
        y=int(np.round(np.sin(angle)*taille))
        if x**2+y**2 == 0:
            if abs(np.cos(angle))>abs(np.sin(angle)):
                x=int(np.sign(np.cos(angle)))
                y=0
            else:
                y=int(np.sign(np.sin(angle)))
                x=0
        rr,cc=line(0,0,y,x)
        rr=rr-rr.min()
        cc=cc-cc.min()
        img=np.zeros((rr.max()+1,cc.max()+1) )
        img[rr,cc]=1
        return img
    raise RuntimeError('Error, form not defined')

    

def sp_noise(image,prob):
    '''
    Add salt and pepper noise to image
    prob: Probability of the noise
    '''
    output = np.zeros(image.shape,np.uint8)
    thres = 1 - prob 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]
    return output
    

### You may need to use subplots. Here is an example

Here we will assume that we will display 3 images, then we define 1 row and 3 columns.

-------------------

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 7))

ax[0].imshow(image, cmap='gray')

ax[0].set_title('your title')

ax[0].axis('off')

ax[1].imshow(image, cmap='gray')

ax[1].set_title('your title')

ax[1].axis('off')

ax[2].imshow(image, cmap='gray')

ax[2].set_title('your title')

ax[2].axis('off')

plt.subplots_adjust()

plt.show()

---------------------

For each image, we have an ax that range from 0 to 2


You can also save the figure and examine it locally using
- fig.savefig("./path/image.png", bbox_inches='tight', dpi=300)


---------------------------------
The Coding and Analysis Session Start Here.
--------------------------------------
----------------------

> <left><b><font style="color: blue" size="3"> Please read carefully before start coding any remark or hint that is given to you.</font></b></center> &ensp;


# Problem 1: Binary Morphological Analysis
- The objective of this problem is to extract the boundary between the small and the larg circles. For that certain morphological operations must be defined.

**1.** Read the input image `large_small_circles.png` stored in the folder `Images`. For that, use the function `skio.imread`. Remember to provide the path`./Images/large_small_circles.png`. **Make sure the folder Images is in the same directory as your .ipyth** otherwise, you have to give the full path.

In [ ]:
img_circle= ##############3

### * Print out the Image Charachteristics, that is, its dimension, min and maximum values.

In [ ]:
print('Our image has a shape of ', img_circle.shape)
print('Its max pixel intensity value is ',img_circle.max())
print('Its min pixel intensity value is ',img_circle.min())

**2. The first goal is to extract small circles and large circles by performing certain Morphological operations.** 

**Therefore, you have to apply different Morphological Operations such opening and/or closing.** 

*For these Morphological Operations, consider the following:*

- **defining structuring elements (SE)** (to be defined using the function `stucturing_element()`, look at the function to know what are the available SE so that you can use them).
    * Example SE = stucturing_element('disk',1) which will give cross-shape SE.
    * You can display it using plt.imshow(SE, cmap='gray')
- **defining the size of SE.** e.g `1,3,5, 7, 9,13,15,17,19,21,23, etc ....`. For that you have to search for the correct size experimentaly and report the best one. Consider the size as the radius in case of disk. Remember, based on the shape and the size, you can eleminate certain shapes.

- If you extacted small/large circles which contain some boundaries of small/large circles that are unwanted, consider filtering them using certain morphological operation.

**Remark:** we imported the library `morphology` from `skimage` and we called it as `morph` therefore, to use dilation for example, use `morph.dilation` or for erosion `morph.erosion` and so on. Same for opening and closing.

**3. Extract the boundaries between the small and the large circles.** 
- For that, think about using the image of the large circle you extacted. 
- Then, make those circles connected as one object using certain morphological operation. 
- Once connected correctly, you can compute the morphological gradient on that image (which is the dilation(input) - erosion(input).
- Once the boundary obtained, superimpose it over the original image.

**4. Comment on the obtained results. Define the role of Morphology in this problem, that is for what purpose has been used?** 

# Problem 2: Image Thresholding
- The objective of this problem is to perform multi otsu thresholding to extract various classes.
- To add noise (salt and pepper, Gaussian noise).
- To perform thresholding over noisy images.
- To filter out the noise using special filters and using Morphological operations.

**1.** Read the input image `Lungs.jpg` stored in the folder `Images` and display it.

**2.** Use the function `threshold_otsu` to compute the global threshold. Then use also the functions `cv2.adaptiveThreshold` and `cv2.adaptiveThreshold`. For those functions, you have to define and experiments several block size until you find the correct one. 


 - Once thresholds obtained, binarize the image. Then use subplot to display the following three elements: 1. The original image over  ax[0], the histogram via ax[1].hist(image.ravel(), bins=255) and the threshold over the histogram via ax[1].axvline(threshold, color='r') and the thresholded image ax[2].
 - Comment on the results, do they varry or they converge to the same results?

**3.** Perform Multi-otsu Thresholding using the function `threshold_multiotsu()`. Here, perform different experimentation by considering different number of classes. For each experiment comment on the obtained results (what is being segmented, its quality, possible overlap with other classes, etc). For each experiment, perform subplot for the image, histogram and thresholded regions.

**4.** Load and display the image `brain.jpg`. Perform multi Otsu with classes=4.
- Use the function sp_noise() with probability = 0.05 to add Salt and Pepper Noise.
- Threshold the image and comment on the results.

**5.** `top-hat transform` is a morphological method that has two operations called:
- `White top-hat transform` which is the difference of the input image and the result after opening morphology operation.
- `Black top-hat transform` which is the difference of the closing morphology operation and input image.

   - **Consider White top-hat transform method and perform some experimentation according to the size and shape of the structuring element.**
   - **Once `top_hat` is obtained, consider computing `noise_img-top_hat`.**
   - **Perform the right morphological operation over `noise_img-top_hat` to reduce the noise.**
   - **Perform Multi Otsu**
   - **Conclude the results**


 


**6.** Consider cleaning the salt and pepper noise by smoothing the image using median filter with a certain size block.

   - Use the function `cv2.medianBlur`
   - What is the effect of the kernel size?
   - Once cleaned, perform again thresholding.
   - Compare the result with those obtained from Morphological operation.

**7.** We will add Gaussian Noise to the image brain_img. Before doing so, 1st we will normalize between 0 and 1. Then we will add certain shape of noise. Once added, we transform the scale back between 0 and 255. **This part is given**

In [ ]:
# Gaussian Noise
noise =  np.random.normal(loc=0, scale=1, size=brain_img.shape)
plt.show()
plt.hist(noise.ravel(), bins=100)
plt.show()

In [ ]:
# Normalize before adding the noise
img = brain_img/255.0
Gaussian_noise = np.clip(np.where(img*2 <= 1, (img*2*(1 + noise*0.2)), (1-img*2+1)*(1 + noise*0.2)*-1 + 2)/2, 0,1)

# Rescale between 0 and 255
Gaussian_noise_img = np.clip(Gaussian_noise*255, 0, 255).astype("uint8")

# Plot the image
plt.imshow(Gaussian_noise_img)
plt.axis('off')

**8.** Perform threshold_multiotsu and adaptiveThreshold using ADAPTIVE_THRESH_MEAN_C and ADAPTIVE_THRESH_GAUSSIAN_C. Comment on the obtained results.

## Remove Gaussian Noise by appling average filter

**9.** Consider using the function `gaussian` with certain sigma to remove Gaussian noise. 
- Display the image and comment on the effect of such filter.
- Perform thresholding again.
- Compare the results with those obtained over the original image and study if any changes or pixel corruption has been resulted.
- Can we do any Gaussian Noise removal using Morphological filters? If yes, show the experiments, If No, comment on why.